# Byte Pair Encoding (BPE) Tokenizer From Scratch -- Simple 从零构建BPE分词器 -- 简易版

 <!--  -This is a standalone notebook implementing the popular byte pair encoding (BPE) tokenization algorithm, which is used in models like GPT-2 to GPT-4, Llama 3, etc., from scratch for educational purposes -->
- 这是一个独立的笔记本，从零开始实现了流行的字节对编码 (BPE) 分词算法，该算法被应用于 GPT-2 到 GPT-4、Llama 3 等模型中，用于学习目的。
 <!--  - For more details about the purpose of tokenization, please refer to [Chapter 2](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/01_main-chapter-code/ch02.ipynb); this code here is bonus material explaining the BPE algorithm-->
- 有关分词目的的更多详细信息，请参考[第二章 ch02.ipynb](../01_main-chapter-code/ch02.ipynb);这段代码是解释 BPE 算法的附加材料
 <!--  - The original BPE tokenizer that OpenAI implemented for training the original GPT models can be found [here](https://github.com/openai/gpt-2/blob/master/src/encoder.py)-->
- OpenAI 为训练原始 GPT 模型而实现的原始 BPE 分词器可以在[这里](https://github.com/openai/gpt-2/blob/master/src/encoder.py)找到
 <!--  - The BPE algorithm was originally described in 1994: "[A New Algorithm for Data Compression](http://www.pennelynn.com/Documents/CUJ/HTML/94HTML/19940045.HTM)" by Philip Gage-->
- BPE 算法最初由 Philip Gage 于 1994 年提出：[一种新的数据压缩算法](http://www.pennelynn.com/Documents/CUJ/HTML/94HTML/19940045.HTM)
 <!--  - Most projects, including Llama 3, nowadays use OpenAI's open-source [tiktoken library](https://github.com/openai/tiktoken) due to its computational performance; it allows loading pretrained GPT-2 and GPT-4 tokenizers, for example (the Llama 3 models were trained using the GPT-4 tokenizer as well)-->
- 由于其计算性能，现在大多数项目（包括 Llama 3）都使用 OpenAI 的开源[tiktoken 库](https://github.com/openai/tiktoken)，它允许加载预训练的 GPT-2 和 GPT-4 分词器，例如（Llama 3 模型也是使用 GPT-4 分词器训练的）。
 <!--  - The difference between the implementations above and my implementation in this notebook, besides it being is that it also includes a function for training the tokenizer (for educational purposes)-->
- 上述实现与我在本笔记本中的实现的区别在于，我的实现还包含一个用于训练分词器的函数（用于学习目的）。
 <!--  - There's also an implementation called [minBPE](https://github.com/karpathy/minbpe) with training support, which is maybe more performant (my implementation here is focused on educational purposes); in contrast to `minbpe` my implementation additionally allows loading the original OpenAI tokenizer vocabulary and merges-->
- 还有一个名为 [minBPE](https://github.com/karpathy/minbpe) 的实现，它支持训练，性能可能更高（我的实现侧重于教学目的）；与 `minbpe` 不同的是，我的实现还允许加载原始的 OpenAI 分词器词汇表并进行合并。

<!-- **This is a very naive implementation for educational purposes. The [bpe-from-scratch.ipynb](bpe-from-scratch.ipynb) notebook contains a more sophisticated (but much harder to read) implementation that matches the behavior in tiktoken.**-->
**这是一个非常简单的教学用途实现。[bpe-from-scratch.ipynb](bpe-from-scratch.ipynb) notebook 中包含一个更复杂（但更难读懂）的实现，它与 tiktoken 的行为相匹配。**

&nbsp;
# 1. <!--The main idea behind byte pair encoding (BPE)--> 字节对编码（BPE）背后的主要思想

<!-- - The main idea in BPE is to convert text into an integer representation (token IDs) for LLM training (see [Chapter 2](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/01_main-chapter-code/ch02.ipynb))-->
- BPE 的主要思想是将文本转换为整数表示（标记 ID），以便进行 LLM 训练.(详情参考[第二章 ch02.ipynb](../01_main-chapter-code/ch02.ipynb))

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/bpe-from-scratch/bpe-overview.webp" width="700px">

&nbsp;
## 1.1 Bits and bytes(比特与字节)

<!-- - Before getting to the BPE algorithm, let's introduce the notion of bytes-->
- 在介绍 BPE 算法之前，我们先来了解一下字节的概念。
<!-- - Consider converting text into a byte array (BPE stands for "byte" pair encoding after all):-->
- 考虑将文本转换为字节数组（毕竟 BPE 代表的是“字节”对编码）：

In [1]:
text = "This is some text"
byte_ary = bytearray(text, "utf-8")
print(byte_ary)

bytearray(b'This is some text')


<!-- - When we call `list()` on a `bytearray` object, each byte is treated as an individual element, and the result is a list of integers corresponding to the byte values:-->
- 当我们对 `bytearray` 对象调用 `list()` 时，每个字节都被视为一个单独的元素，结果是一个与字节值对应的整数列表：

In [3]:
ids = list(byte_ary)
print(ids)

[84, 104, 105, 115, 32, 105, 115, 32, 115, 111, 109, 101, 32, 116, 101, 120, 116]


<!-- - This would be a valid way to convert text into a token ID representation that we need for the embedding layer of an LLM
- However, the downside of this approach is that it is creating one ID for each character (that's a lot of IDs for a short text!)
- I.e., this means for a 17-character input text, we have to use 17 token IDs as input to the LLM:-->
- 这是一种将文本转换成LLM嵌入层所需的token ID表示有效方法。然而，这种方法的缺点是他为每个字符创建一个ID(对于短文本身来说，这将生成大量ID),
- 也就是说，对应17个字符的输入文本，我们所需使用的17个token ID作为LLM的输入。

In [4]:
print("Number of characters:", len(text))
print("Number of token IDs:", len(ids))

Number of characters: 17
Number of token IDs: 17


<!-- - If you have worked with LLMs before, you may know that the BPE tokenizers have a vocabulary where we have a token ID for whole words or subwords instead of each character
- For example, the GPT-2 tokenizer tokenizes the same text ("This is some text") into only 4 instead of 17 tokens: `1212, 318, 617, 2420`
- You can double-check this using the interactive [tiktoken app](https://tiktokenizer.vercel.app/?model=gpt2) or the [tiktoken library](https://github.com/openai/tiktoken):-->
- 如果您之前使用过 LLM，您可能知道 BPE 分词器有一个词汇表，其中我们为整个单词或子词分配一个词元 ID，而不是为每个字符分配一个词元 ID。
- 例如，GPT-2分词器将相同的文本("This is some text") 分成只有4个次元而不是17个次元ID:`1212, 318, 617, 2420`
- 你可以用交互[token软件](https://tiktokenizer.vercel.app/?model=gpt2)或者[token库](https://github.com/openai/tiktoken)进行二次验证：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/bpe-from-scratch/tiktokenizer.webp" width="800px">

```python
import tiktoken

gpt2_tokenizer = tiktoken.get_encoding("gpt2")
gpt2_tokenizer.encode("This is some text")
# prints [1212, 318, 617, 2420]
```

In [7]:
import tiktoken

gpt2_tokenizer = tiktoken.get_encoding("gpt2")
gpt2_tokenizer.encode("This is some text")
# prints( [1212, 318, 617, 2420])

[1212, 318, 617, 2420]

<!-- - Since a byte consists of 8 bits, there are 2<sup>8</sup> = 256 possible values that a single byte can represent, ranging from 0 to 255
- You can confirm this by executing the code `bytearray(range(0, 257))`, which will warn you that `ValueError: byte must be in range(0, 256)`)
- A BPE tokenizer usually uses these 256 values as its first 256 single-character tokens; one could visually check this by running the following code:-->
- 由于一个字节由 8 位组成，因此一个字节可以表示 2⁸ = 256 个可能的值，范围从 0 到 255。您可以通过执行代码 `bytearray(range(0, 257))` 来验证这一点，
- 该代码会警告您 `ValueError: byte must be in range(0, 256)`。BPE 分词器通常使用这 256 个值作为其前 256 个单字符标记；您可以通过运行以下代码来直观地检查这一点：
```python
import tiktoken
gpt2_tokenizer = tiktoken.get_encoding("gpt2")

for i in range(300):
    decoded = gpt2_tokenizer.decode([i])
    print(f"{i}: {decoded}")
"""
prints:
0: !
1: "
2: #
...
255: �  # <---- single character tokens up to here
256:  t
257:  a
...
298: ent
299:  n
"""
```

In [1]:
import tiktoken
gpt2_tokenizer = tiktoken.get_encoding("gpt2")

for i in range(300):
    decoded = gpt2_tokenizer.decode([i])
    print(f"{i}: {decoded}")

0: !
1: "
2: #
3: $
4: %
5: &
6: '
7: (
8: )
9: *
10: +
11: ,
12: -
13: .
14: /
15: 0
16: 1
17: 2
18: 3
19: 4
20: 5
21: 6
22: 7
23: 8
24: 9
25: :
26: ;
27: <
28: =
29: >
30: ?
31: @
32: A
33: B
34: C
35: D
36: E
37: F
38: G
39: H
40: I
41: J
42: K
43: L
44: M
45: N
46: O
47: P
48: Q
49: R
50: S
51: T
52: U
53: V
54: W
55: X
56: Y
57: Z
58: [
59: \
60: ]
61: ^
62: _
63: `
64: a
65: b
66: c
67: d
68: e
69: f
70: g
71: h
72: i
73: j
74: k
75: l
76: m
77: n
78: o
79: p
80: q
81: r
82: s
83: t
84: u
85: v
86: w
87: x
88: y
89: z
90: {
91: |
92: }
93: ~
94: �
95: �
96: �
97: �
98: �
99: �
100: �
101: �
102: �
103: �
104: �
105: �
106: �
107: �
108: �
109: �
110: �
111: �
112: �
113: �
114: �
115: �
116: �
117: �
118: �
119: �
120: �
121: �
122: �
123: �
124: �
125: �
126: �
127: �
128: �
129: �
130: �
131: �
132: �
133: �
134: �
135: �
136: �
137: �
138: �
139: �
140: �
141: �
142: �
143: �
144: �
145: �
146: �
147: �
148: �
149: �
150: �
151: �
152: �
153: �
154: �
155: �
156: �
157: �
158:

<!-- - Above, note that entries 256 and 257 are not single-character values but double-character values (a whitespace + a letter), which is a little shortcoming of the original GPT-2 BPE Tokenizer (this has been improved in the GPT-4 tokenizer)-->
- 如上所述，请注意第 256 和 257 行不是单字符值，而是双字符值（一个空格 + 一个字母），这是原始 GPT-2 BPE 分词器的一个小缺陷（GPT-4 分词器已经改进了这一点）。

&nbsp;
## 1.2 Building the vocabulary

- The goal of the BPE tokenization algorithm is to build a vocabulary of commonly occurring subwords like `298: ent` (which can be found in *entangle, entertain, enter, entrance, entity, ...*, for example), or even complete words like 

```
318: is
617: some
1212: This
2420: text
```

- The BPE algorithm was originally described in 1994: "[A New Algorithm for Data Compression](http://www.pennelynn.com/Documents/CUJ/HTML/94HTML/19940045.HTM)" by Philip Gage
- Before we get to the actual code implementation, the form that is used for LLM tokenizers today can be summarized as follows:

&nbsp;
## 1.3 BPE algorithm outline(BPE算法概述)

**1. Identify frequent pairs(识别频繁配对)**
- In each iteration, scan the text to find the most commonly occurring pair of bytes (or characters)
- 在每次迭代中，扫描文本以找到出现频率最高的字节（或字符）对。

**2. Replace and record(替换并记录)**

- Replace that pair with a new placeholder ID (one not already in use, e.g., if we start with 0...255, the first placeholder would be 256)
- 将该对替换为一个新的占位符 ID（尚未使用的 ID，例如，如果我们从 0...255 开始，则第一个占位符将是 256）。
- Record this mapping in a lookup table
- 在一个查找表中记录这个映射关系
- The size of the lookup table is a hyperparameter, also called "vocabulary size" (for GPT-2, that's
50,257)
- 查找表的大小是一个超参数，也称为“词汇表大小”（对于 GPT-2，该值为：50,257）

**3. Repeat until no gains(重复此步骤直至不再产生收益)**

- Keep repeating steps 1 and 2, continually merging the most frequent pairs
- 保持重复步骤1和步骤2，不断合并出现频率最高的配对
- Stop when no further compression is possible (e.g., no pair occurs more than once)
- 当无法进一步压缩时停止（例如，没有一对元素出现超过一次）。

**Decompression (decoding)(解压缩（解码）)**

- To restore the original text, reverse the process by substituting each ID with its corresponding pair, using the lookup table
- 利用反向操作将使用查找表将每个 ID 替换为其对应的ID对来恢复原文。

&nbsp;
## 1.4 BPE algorithm example (BPE算法示例)

### 1.4.1 Concrete example of the encoding part (steps 1 & 2)(编码部分的具体示例（步骤 1 和 2）)

- Suppose we have the text (training dataset) `the cat in the hat` from which we want to build the vocabulary for a BPE tokenizer
- 假设我们有文本（训练数据集）“戴帽子的猫”，我们想用它构建 BPE 分词器的词汇表。

**Iteration 1(迭代 1)**

1. Identify frequent pairs(识别频繁配对)
  - In this text, "th" appears twice (at the beginning and before the second "e")
  - 在这段文字中，“th”出现了两次（一次在开头，一次在第二个“e”之前）。

2. Replace and record(替换并记录)
  - replace "th" with a new token ID that is not already in use, e.g., 256
  - 将“th”替换为尚未使用的新令牌 ID，例如 256。
  - the new text is: `<256>e cat in <256>e hat`
  - 新文本为：`<256>e cat in <256>e hat`
  - the new vocabulary is
  - 新词汇是

```
  0: ...
  ...
  256: "th"
```

**Iteration 2(迭代2)**

1. **Identify frequent pairs(识别频繁配对)**  
   - In the text `<256>e cat in <256>e hat`, the pair `<256>e` appears twice
   - 在文本 `<256>e cat in <256>e hat`中， `<256>e`这对数字出现了两次

2. **Replace and record(替换并记录)**  
   - replace `<256>e` with a new token ID that is not already in use, for example, `257`.
   - 将`<256>e`替换为未使用的新tokenID，例如`257`
   - The new text is:
   - 新的文本是
     ```
     <257> cat in <257> hat
     ```
   - The updated vocabulary is:
   - 更新后的词汇为
     ```
     0: ...
     ...
     256: "th"
     257: "<256>e"
     ```

**Iteration 3(迭代3)**

1. **Identify frequent pairs(识别频繁配对)**  
   - In the text `<257> cat in <257> hat`, the pair `<257> ` appears twice (once at the beginning and once before “hat”).
   - 在文本 `<257> cat in <257> hat`中， `<257> `这对符号出现了两次（一次在开头，一次在“hat”之前）。

2. **Replace and record(替换并记录)**  
   - replace `<257> ` with a new token ID that is not already in use, for example, `258`.
   - 将 `<257>` 替换为尚未使用的新令牌 ID，例如 `258`。
   - the new text is:
   - 新文本是
     ```
     <258>cat in <258>hat
     ```
   - The updated vocabulary is:
   - 更新后的词汇为
     ```
     0: ...
     ...
     256: "th"
     257: "<256>e"
     258: "<257> "
     ```
     
- and so forth
- 等等

&nbsp;
### 1.4.2 Concrete example of the decoding part (steps 3)(解码部分的具体示例（步骤 3）)

- To restore the original text, we reverse the process by substituting each token ID with its corresponding pair in the reverse order they were introduced
- 为了恢复原文，我们反向操作，将每个词元 ID 替换为其对应的词元对，替换顺序与它们最初出现的顺序相反。
- Start with the final compressed text: `<258>cat in <258>hat`
- 从最终的压缩文本开始：`<258>cat in <258>hat`
-  Substitute `<258>` → `<257> `: `<257> cat in <257> hat`
-  代替 `<258>` → `<257> `: `<257> cat in <257> hat`
- Substitute `<257>` → `<256>e`: `<256>e cat in <256>e hat`
- 代替 `<257>` → `<256>e`: `<256>e cat in <256>e hat`
- Substitute `<256>` → "th": `the cat in the hat`
- 代替`<256>` → "th": `the cat in the hat`

&nbsp;
## 2. A simple BPE implementation(一个简单的BPE实现)

<!-- - Below is an implementation of this algorithm described above as a Python class that mimics the `tiktoken` Python user interface
- Note that the encoding part above describes the original training step via `train()`; however, the `encode()` method works similarly (although it looks a bit more complicated because of the special token handling):-->
- 以下是上述算法的 Python 类实现，它模仿了 `tiktoken` Python 用户界面。
- 请注意，上面的编码部分描述了通过 `train()` 进行的原始训练步骤；但是，`encode()` 方法的工作方式类似（尽管由于特殊的标记处理，它看起来稍微复杂一些）：

<!-- 1. Split the input text into individual bytes
2. Repeatedly find & replace (merge) adjacent tokens (pairs) when they match any pair in the learned BPE merges (from highest to lowest "rank," i.e., in the order they were learned)
3. Continue merging until no more merges can be applied
4. The final list of token IDs is the encoded output -->
1. 将输入文本分割成单个字节
2. 重复查找并替换（合并）相邻的词元（词元对），只要它们与已学习的 BPE 合并结果中的任何词元对匹配即可（从最高“排名”到最低，即按照学习顺序）
3. 继续合并，直到无法再进行合并为止
4. 最终的词元 ID 列表即为编码后的输出

In [2]:
from collections import Counter, deque
from functools import lru_cache


class BPETokenizerSimple:
    def __init__(self):
        # 将 token_id 映射到 token_str
        self.vocab = {}
        # 将 token_str 映射到 token_id
        self.inverse_vocab = {}
        # BPE 合并字典             ：{(token_id1, token_id2)        : merged_token_id}
        self.bpe_merges = {}

    def train(self, text, vocab_size, allowed_special={"<|endoftext|>"}):
        """
        从头开始训练 BPE 分词器。
        参数：
            text（字符串）：训练文本。
            vocab_size（整数）：所需的词汇表大小。
            allowed_special（集合）：要包含的特殊词元集合。
        """
        
        # 预处理：将空格替换为 'Ġ'
        # 注意：Ġ 是 GPT-2 BPE 实现的特殊之处
        # 例如，“Hello world” 可能被分词为 ["Hello", "Ġworld"]
        # （GPT-4 BPE 会将其分词为 ["Hello", " world"]）
        processed_text = []
        for i, char in enumerate(text):
            if char == " " and i != 0:
                processed_text.append("Ġ")
            if char != " ":
                processed_text.append(char)
        processed_text = "".join(processed_text)

        # 使用唯一字符初始化词汇表，如果存在，则包含字母“Ġ”
        # 从前 256 个 ASCII 字符开始
        unique_chars = [chr(i) for i in range(256)]

        # 将 processed_text 中尚未包含的字符添加到 unique_chars 中。
        unique_chars.extend(char for char in sorted(set(processed_text)) if char not in unique_chars)

        # （可选）如果与文本处理相关，请确保包含“Ġ”。
        if 'Ġ' not in unique_chars:
            unique_chars.append('Ġ')

        # 现在创建词汇表和逆词汇表
        self.vocab = {i: char for i, char in enumerate(unique_chars)}
        self.inverse_vocab = {char: i for i, char in self.vocab.items()}

        # 添加允许的特殊标记
        if allowed_special:
            for token in allowed_special:
                if token not in self.inverse_vocab:
                    new_id = len(self.vocab)
                    self.vocab[new_id] = token
                    self.inverse_vocab[token] = new_id

        # 将处理后的文本分词为标记 ID
        token_ids = [self.inverse_vocab[char] for char in processed_text]

        # BPE步骤1-3：反复查找并替换频繁配对
        for new_id in range(len(self.vocab), vocab_size):
            pair_id = self.find_freq_pair(token_ids, mode="most")
            if pair_id is None:  #没有更多配对可以合并了。停止训练。
                break
            token_ids = self.replace_pair(token_ids, pair_id, new_id)
            self.bpe_merges[pair_id] = new_id

        # 使用合并的词元构建词汇表
        for (p0, p1), new_id in self.bpe_merges.items():
            merged_token = self.vocab[p0] + self.vocab[p1]
            self.vocab[new_id] = merged_token
            self.inverse_vocab[merged_token] = new_id

    def encode(self, text):
        """
        将输入文本编码为标记 ID 列表。
        参数：
            text（字符串）：要编码的文本。
        返回值：
            List[int]：标记 ID 列表。
        """
        tokens = []
        # 将文本分割成词元，并保留换行符。
        words = text.replace("\n", " \n ").split()  #确保将换行符“\n”视为单独的标记。

        for i, word in enumerate(words):
            if i > 0 and not word.startswith("\n"):
                tokens.append("Ġ" + word)  # 在空格或换行符后的单词前添加“Ġ”。
            else:
                tokens.append(word)  # 处理第一个单词或单独的换行符'\n'

        token_ids = []
        for token in tokens:
            if token in self.inverse_vocab:
                # token 包含在词汇表中。
                token_id = self.inverse_vocab[token]
                token_ids.append(token_id)
            else:
                # 尝试通过 BPE 处理子词分词。
                sub_token_ids = self.tokenize_with_bpe(token)
                token_ids.extend(sub_token_ids)

        return token_ids

    def tokenize_with_bpe(self, token):
        """

        使用 BPE 合并对单个词元进行分词。
        参数：
            token (str)：要进行分词的词元。
        返回值：
            List[int]：应用 BPE 后的词元 ID 列表。
        """
        # 将标记分解成单个字符（作为初始标记 ID）
        token_ids = [self.inverse_vocab.get(char, None) for char in token]
        if None in token_ids:
            missing_chars = [char for char, tid in zip(token, token_ids) if tid is None]
            raise ValueError(f"Characters not found in vocab: {missing_chars}")

        can_merge = True
        while can_merge and len(token_ids) > 1:
            can_merge = False
            new_tokens = []
            i = 0
            while i < len(token_ids) - 1:
                pair = (token_ids[i], token_ids[i + 1])
                if pair in self.bpe_merges:
                    merged_token_id = self.bpe_merges[pair]
                    new_tokens.append(merged_token_id)
                    # 取消注释仅供学习目的：
                    print(f"Merged pair {pair} -> {merged_token_id} ('{self.vocab[merged_token_id]}')")
                    i += 2  # 跳过下一个令牌，因为它已被合并。
                    can_merge = True
                else:
                    new_tokens.append(token_ids[i])
                    i += 1
            if i < len(token_ids):
                new_tokens.append(token_ids[i])
            token_ids = new_tokens

        return token_ids

    def decode(self, token_ids):
        """
        将令牌 ID 列表解码回字符串。

        参数：
            token_ids (List[int]): 要解码的令牌 ID 列表。

        返回值：

            str: 解码后的字符串。
        """
        decoded_string = ""
        for token_id in token_ids:
            if token_id not in self.vocab:
                raise ValueError(f"Token ID {token_id} not found in vocab.")
            token = self.vocab[token_id]
            if token.startswith("Ġ"):
                #  空格替换成'Ġ'
                decoded_string += " " + token[1:]
            else:
                decoded_string += token
        return decoded_string

    @lru_cache(maxsize=None)
    def get_special_token_id(self, token):
        return self.inverse_vocab.get(token, None)

    @staticmethod
    def find_freq_pair(token_ids, mode="most"):
        pairs = Counter(zip(token_ids, token_ids[1:]))

        if mode == "most":
            return max(pairs.items(), key=lambda x: x[1])[0]
        elif mode == "least":
            return min(pairs.items(), key=lambda x: x[1])[0]
        else:
            raise ValueError("Invalid mode. Choose 'most' or 'least'.")

    @staticmethod
    def replace_pair(token_ids, pair_id, new_id):
        dq = deque(token_ids)
        replaced = []

        while dq:
            current = dq.popleft()
            if dq and (current, dq[0]) == pair_id:
                replaced.append(new_id)
                # Remove the 2nd token of the pair, 1st was already removed
                dq.popleft()
            else:
                replaced.append(current)

        return replaced


<!-- - There is a lot of code in the `BPETokenizerSimple` class above, and discussing it in detail is out of scope for this notebook, but the next section offers a short overview of the usage to understand the class methods a bit better -->
上面的 `BPETokenizerSimple` 类包含大量代码，详细讨论超出了本笔记本的范围，但下一节将简要概述其用法，以便更好地理解类方法。

## 3. BPE implementation walkthrough(BPE实施流程概述)

<!-- - In practice, I highly recommend using [tiktoken](https://github.com/openai/tiktoken) as my implementation above focuses on readability and educational purposes, not on performance-->
- 实际上，我强烈建议使用 [tiktoken](https://github.com/openai/tiktoken)，因为我上面的实现侧重于可读性和教育目的，而非性能。
<!-- - However, the usage is more or less similar to tiktoken, except that tiktoken does not have a training method -->
- 然而，它的使用方法与 tiktoken 大体相似，只是 tiktoken 没有训练方法。
<!-- - Let's see how my `BPETokenizerSimple` Python code above works by looking at some examples below (a detailed code discussion is out of scope for this notebook) -->
- 让我们通过下面的一些示例来了解一下我上面的 `BPETokenizerSimple` Python 代码是如何工作的（详细的代码讨论超出了本笔记本的范围）。

### 3.1 Training, encoding, and decoding(训练，编码和解码)

<!-- - First, let's consider some sample text as our training dataset: -->
- 首先，我们考虑一些示例文本作为训练数据集：

In [9]:
import os
import urllib.request

if not os.path.exists("../01_main-chapter-code/the-verdict.txt"):
    print("not has the-verdict.txt")
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "../01_main-chapter-code/the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)
else:
    print("has the-verdict.txt")
with open("../01_main-chapter-code/the-verdict.txt", "r", encoding="utf-8") as f: # 添加 ../01_main-chapter-code/
    text = f.read()

has the-verdict.txt


<!-- - Next, let's initialize and train the BPE tokenizer with a vocabulary size of 1,000 -->
- 接下来，我们初始化并训练词汇量为 1000 的 BPE 分词器。
<!-- - Note that the vocabulary size is already 255 by default due to the byte values discussed earlier, so we are only "learning" 745 vocabulary entries -->
- 请注意，由于前面讨论的字节值，词汇表大小默认已为 255，因此我们实际上只“学习”了 745 个词汇条目。
<!-- - For comparison, the GPT-2 vocabulary is 50,257 tokens, the GPT-4 vocabulary is 100,256 tokens (`cl100k_base` in tiktoken), and GPT-4o uses 199,997 tokens (`o200k_base` in tiktoken); they have all much bigger training sets compared to our simple example text above -->
- 作为对比，GPT-2 的词汇表包含 50,257 个词元，GPT-4 的词汇表包含 100,256 个词元（在 tiktoken 中称为 `cl100k_base`），而 GPT-4o 则使用 199,997 个词元（在 tiktoken 中称为 `o200k_base`）；它们的训练集都比我们上面简单的示例文本要大得多。

In [12]:
tokenizer = BPETokenizerSimple()
tokenizer.train(text, vocab_size=1000, allowed_special={"<|endoftext|>"})

<!-- - You may want to inspect the vocabulary contents (but note it will create a long list) -->
- 您可能需要查看词汇表内容（但请注意，这将生成一个很长的列表）。

In [13]:
# print(tokenizer.vocab)
print(len(tokenizer.vocab))

1000


<!-- - This vocabulary is created by merging 742 times (~ `1000 - len(range(0, 256))`) -->
- 该词汇表是通过合并 742 次创建的(~ `1000 - len(range(0, 256))`)。

In [14]:
print(len(tokenizer.bpe_merges))

742


<!-- - This means that the first 256 entries are single-character tokens -->
- 这意味着前 256 个条目都是单字符标记。

<!-- - Next, let's use the created merges via the `encode` method to encode some text: -->
- 接下来，让我们使用 encode 方法对创建的合并结果进行编码：

In [16]:
input_text = "Jack embraced beauty through art and life."
token_ids = tokenizer.encode(input_text)
print(token_ids)

Merged pair (101, 109) -> 654 ('em')
Merged pair (98, 114) -> 531 ('br')
Merged pair (97, 99) -> 302 ('ac')
Merged pair (101, 100) -> 311 ('ed')
Merged pair (98, 101) -> 296 ('be')
Merged pair (117, 116) -> 465 ('ut')
Merged pair (256, 97) -> 287 ('Ġa')
Merged pair (287, 114) -> 841 ('Ġar')
Merged pair (256, 97) -> 287 ('Ġa')
Merged pair (110, 100) -> 466 ('nd')
Merged pair (108, 105) -> 326 ('li')
Merged pair (102, 101) -> 972 ('fe')
[424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46]


In [17]:
print("Number of characters:", len(input_text))
print("Number of token IDs:", len(token_ids))

Number of characters: 42
Number of token IDs: 20


<!-- - From the lengths above, we can see that a 42-character sentence was encoded into 20 token IDs, effectively cutting the input length roughly in half compared to a character-byte-based encoding -->
- 从上述长度可以看出，一个 42 个字符的句子被编码成 20 个标记 ID，与基于字符-字节的编码相比，输入长度有效地减少了大约一半。

<!-- - Note that the vocabulary itself is used in the `decode()` method, which allows us to map the token IDs back into text: -->
- 请注意，词汇表本身在 `decode()` 方法中使用，这使我们能够将标记 ID 映射回文本：

In [18]:
print(token_ids)

[424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46]


In [19]:
print(tokenizer.decode(token_ids))

Jack embraced beauty through art and life.


<!-- - Iterating over each token ID can give us a better understanding of how the token IDs are decoded via the vocabulary: -->
- 遍历每个词元 ID 可以让我们更好地理解词元 ID 如何通过词汇表进行解码：

In [20]:
for token_id in token_ids:
    print(f"{token_id} -> {tokenizer.decode([token_id])}")

424 -> Jack
256 ->  
654 -> em
531 -> br
302 -> ac
311 -> ed
256 ->  
296 -> be
97 -> a
465 -> ut
121 -> y
595 ->  through
841 ->  ar
116 -> t
287 ->  a
466 -> nd
256 ->  
326 -> li
972 -> fe
46 -> .


<!-- - As we can see, most token IDs represent 2-character subwords; that's because the training data text is very short with not that many repetitive words, and because we used a relatively small vocabulary size -->
- 我们可以看到，大多数词元 ID 代表的是双字符子词；这是因为训练数据文本很短，重复词不多，而且我们使用的词汇量也相对较小。

<!-- - As a summary, calling `decode(encode())` should be able to reproduce arbitrary input texts: -->
- 总而言之，调用 `decode(encode())` 应该能够重现任意输入的文本：

In [21]:
tokenizer.decode(tokenizer.encode("This is some text."))

Merged pair (84, 104) -> 542 ('Th')
Merged pair (105, 115) -> 299 ('is')
Merged pair (105, 115) -> 299 ('is')
Merged pair (256, 115) -> 321 ('Ġs')
Merged pair (111, 109) -> 305 ('om')
Merged pair (256, 116) -> 259 ('Ġt')
Merged pair (101, 120) -> 461 ('ex')


'This is some text.'

&nbsp;
# 4. Conclusion 结论

<!-- - That's it! That's how BPE works in a nutshell, complete with a training method for creating new tokenizers -->
- 就是这样！这就是BPE的简要工作原理，其中包含了创建新分词器的训练方法。
<!-- - I hope you found this brief tutorial useful for educational purposes; if you have any questions, please feel free to open a new Discussion [here](https://github.com/rasbt/LLMs-from-scratch/discussions/categories/q-a) -->


<!-- **This is a very naive implementation for educational purposes. The [bpe-from-scratch.ipynb](bpe-from-scratch.ipynb) notebook contains a more sophisticated (but much harder to read) implementation that matches the behavior in tiktoken.** -->
**这是一个非常简单的教学用途实现。[bpe-from-scratch.ipynb](bpe-from-scratch.ipynb) notebook 中包含一个更复杂（但更难读懂）的实现，它与 tiktoken 的行为相匹配**